In [44]:
import networkx as nx
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go


In [45]:
def create_graph(positions, edges):
    G = nx.Graph()
    G.add_nodes_from(positions.keys())
    G.add_edges_from(edges)

    for n, p in positions.items():
        G.nodes[n]["pos"] = p
    return G


In [46]:
def load_graph(path):
    positions = dict()
    edges = []
    flag = None
    with open(path, "r") as f:
        idx = 0
        for line in f:
            line = line.strip()
            if line == "a" or line == "b":
                flag = line
                continue

            if flag == "a":
                positions[idx] = list(map(float, line.split()))
                idx += 1
            else:
                edges.append(list(map(int, line.split())))

    assert len(positions.keys()) == len(edges) + 1

    return create_graph(positions, edges)


In [47]:
def get_edge_trace(G, color="#888"):
    edge_x = []
    edge_y = []
    for u, v in G.edges():
        pos_u = G.nodes[u]["pos"]
        pos_v = G.nodes[v]["pos"]
        edge_x.extend([pos_u[0], pos_v[0], None])
        edge_y.extend([pos_u[1], pos_v[1], None])

    edge_trace = go.Scatter(
        x=edge_x,
        y=edge_y,
        line=dict(width=0.5, color=color),
        mode="lines",
        showlegend=False,
    )
    return edge_trace


def get_node_trace(G, color="#0000FF", size=10, name="Original"):
    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = G.nodes[node]["pos"]
        node_x.append(x)
        node_y.append(y)

    node_trace = go.Scatter(
        x=node_x,
        y=node_y,
        mode="markers",
        name=name,
        marker=dict(
            color=color,
            size=size,
        ),
    )
    return node_trace


In [48]:
def plot_graph(g1, g2):
    e1 = get_edge_trace(g1)
    e2 = get_edge_trace(g2, color="#000")
    n1 = get_node_trace(g1)
    n2 = get_node_trace(g2, color="#FF0000", size=6, name="Optimized")
    
    fig = go.Figure(
        data=[e1, n1, e2, n2],
        layout=go.Layout(
            title="Original and Optimized Graphs",
        ),
    )
    fig.show()


In [51]:
path = "../graphs/graph_01.txt"
g1 = load_graph(path)

# Get the positions of each node
pos = nx.get_node_attributes(g1, "pos")
num_nodes = g1.number_of_nodes()

# Create the optimization variables
opt_path = cp.Variable((num_nodes, 2))

# Define the regularization parameter
alpha = 5

# Define the objective function to minimize the curvature and distance
obj = cp.sum_squares(
    opt_path[1:-1, :] - 0.5 * (opt_path[0:-2, :] + opt_path[2:, :])
) + alpha * cp.sum_squares(opt_path - np.array(list(pos.values())))

# Define the constraints for fixed nodes
constraints = []
for node, position in pos.items():
    if len(list(g1.neighbors(node))) != 2:
        constraints.append(opt_path[node, :] == np.array(position))

# Create the optimization problem
problem = cp.Problem(cp.Minimize(obj), constraints)

# Solve the problem
problem.solve()

# Retrieve the optimized path
optimized_pos = {node: tuple(opt_path[node, :].value) for node in g1.nodes()}

# Create optimized graph
g2 = create_graph(optimized_pos, g1.edges())

# Draw the original and optimized graphs together
plot_graph(g1, g2)
